In [1]:
%load_ext line_profiler

In [2]:
import os
import time

In [3]:
import numpy as np
from scipy.spatial.distance import pdist

In [4]:
import numba
from numba import jit, njit, vectorize, guvectorize

In [5]:
import bokeh.plotting
import bokeh.io
bokeh.io.output_file('pr.html')

In [6]:
from bokeh.palettes import Dark2_7 as palette

In [7]:
coor_fname = '/some/path/nl1_nrx1b.coor'
print(os.path.splitext(coor_fname)[0])

/some/path/nl1_nrx1b


In [8]:
os.path.split(coor_fname)

('/some/path', 'nl1_nrx1b.coor')

In [9]:
coor_fname = 'nl1_nrx1b.coor'
coor = np.loadtxt(coor_fname, delimiter=',')
out_fname = '{}.pr'.format(os.path.splitext(coor_fname))

In [10]:
def calc_pr_hist(coor):
    # calculate the euclidean distances
    dist = pdist(coor)

    # bin the distances into P(r)
    r_max = dist.max()

    n_bins = np.ceil(r_max).astype(int) + 1
    bin_edges = np.arange(n_bins + 1) - 0.5
    pr = np.empty([n_bins, 2])
    pr[:, 0] = np.arange(n_bins) + 1
    pr[:, 1], _ = np.histogram(dist, bin_edges)
    return pr

In [25]:
pr0 = calc_pr_hist(coor)

In [12]:
%lprun -f calc_pr_hist calc_pr_hist(coor)

On odin, the histogram function took 13.98 s, or 90.0% of the total time.

In [13]:
def calc_pr_bin(coor):
    # calculate the euclidean distances
    dist = pdist(coor)

    # bin the distances into P(r)
    r_max = dist.max()

    n_bins = np.ceil(r_max).astype(int) + 1
    pr = np.empty([n_bins, 2], dtype=np.int)
    pr[:, 0] = np.arange(n_bins) + 1
    int_dist = np.round(dist).astype(np.int)
    pr[:, 1] = np.bincount(int_dist)
    return pr

In [27]:
pr1 = calc_pr_bin(coor)

In [15]:
%lprun -f calc_pr_bin calc_pr_bin(coor)

On odin, the round function took 2.2 s, 50.1 %, and the bincount function took 0.5 s, 11.5 %.  Total this is 2.7 s or 62.6% of the total time.

In [16]:
@jit(['int64[:], int64[:]',
      'float32[:], int64[:]',
      'float64[:], int64[:]'],
     nopython=True)
def jit_hist(distances, pr):
    for dist in distances:
        pr[round(dist)] += 1

In [17]:
def jit_calc_pr(coor):
    # calculate the euclidean distances
    dist = pdist(coor)

    # bin the distances into P(r)
    r_max = dist.max()
    n_bins = np.round(r_max).astype(int) + 1
    pr = np.zeros([n_bins, 2], dtype=np.int)
    pr[:, 0] = np.arange(n_bins) + 1
    jit_hist(dist, pr[:, 1])
    return pr

In [18]:
pr2 = jit_calc_pr(coor)

In [19]:
%timeit pr2 = jit_calc_pr(coor)

1 loop, best of 3: 1.97 s per loop


In [20]:
%lprun -f jit_calc_pr jit_calc_pr(coor)

On odin, the histogram function took 45.9 s, or 22.9% of the total time.

In [28]:
p = bokeh.plotting.figure()
p.square(pr0[:, 0], pr0[:, 1], color=palette[0], legend='np.histogram')
p.circle(pr1[:, 0], pr1[:, 1] * 2, color=palette[1], legend='np.bincount')
p.circle(pr2[:, 0], pr2[:, 1] * 3, color=palette[2], legend='numba.jit')

bokeh.io.show(p)

In [ ]:

@numba.jit(['int64[:], int64[:]',
      'float32[:], int64[:]',
      'float64[:], int64[:]'],
     nopython=True)
def jit_hist(distances, pr):
    for dist in distances:
        pr[round(dist)] += 1


def calc_pr_numba(coor):
    '''
    calculate P(r) from an array of coordinates
    when written, this was twice as fast as python method
    '''
    # calculate the euclidean distances
    dist = pdist(coor)

    # bin the distances into P(r)
    r_max = dist.max()
    n_bins = np.round(r_max).astype(int) + 1
    pr = np.zeros([n_bins, 2], dtype=np.int)
    pr[:, 0] = np.arange(n_bins) + 1
    jit_histogram(dist, pr[:, 1])

    return pr